In [ ]:
library(BRVM)
library(IRkernel)
library(IRdisplay)
library(repr)
library(dplyr)
library(formattable)
library(scales)
library(readr)
library(rvest)
library(openxlsx)

In [ ]:
# Extract BRVM Info with BRVM_ticker_desc
brvm_info <- BRVM_ticker_desc()

# Extract tickers from ticker_info
tickers <- brvm_info$Ticker

In [ ]:
# Retrieve BRVM information for the specified tickers
# from January 1st, 2023, to yesterday's date
brvm_info_list <- BRVM_get(.symbol = tickers, .from = "2023-01-01", .to = Sys.Date() - 1)

In [ ]:
# Merge the BRVM information retrieved for the specified tickers
# with additional BRVM information based on the common column "Ticker"
BRVM_frame <- merge(brvm_info_list, brvm_info, by = "Ticker")

In [ ]:
 # Display a message indicating the process of loading BRVM Market Cap information
cat("Loading BRVM Market Cap Info\n")

# Load BRVM market cap data
B_Market <- BRVM_cap()

# Rename the columns
B_Market <- B_Market %>%
  rename(Ticker = Symbol,
         `Market Cap` = `Global capitalization`,
         `Market Cap (%)` = `Global capitalization (%)`) %>%
  select(Ticker, `Market Cap`, `Market Cap (%)`)

# Format the "Market Cap" column as currency
B_Market$`Market Cap` <- currency(B_Market$`Market Cap`)

# Sort the data frame by "Market Cap" column in descending order
B_Market <- arrange(B_Market, desc(`Market Cap`))

# Format the "Market Cap (%)" column as percentage
B_Market$`Market Cap (%)` <- as.numeric(B_Market$`Market Cap (%)`)

# Print the modified data
print(B_Market)

In [ ]:
# Merge BRVM dataframes by Ticker
# Rename Company name column to Company
# Select relevant columns for BRVM analysis
# Arrange data by Market Cap in descending order
BRVM_columns <- merge(BRVM_frame, B_Market, by = "Ticker") %>%
  rename(Company = `Company name`) %>%
  select(Ticker,Sector, Company, Country, Date, Open, High, Low, Close, Volume, `Market Cap`, `Market Cap (%)`) %>%
  arrange(desc(`Market Cap`))

# Format Market Cap column to display values in T, B, or M
BRVM_columns$`Market Cap` <- case_when(
  BRVM_columns$`Market Cap` >= 1e12 ~ sprintf("$%sT", comma(BRVM_columns$`Market Cap` / 1e12)),
  BRVM_columns$`Market Cap` >= 1e9 ~ sprintf("$%sB", comma(BRVM_columns$`Market Cap` / 1e9)),
  BRVM_columns$`Market Cap` >= 1e6 ~ sprintf("$%sM", comma(BRVM_columns$`Market Cap` / 1e6))
)

# Format Market Cap (%) column as percentage
BRVM_columns$`Market Cap (%)` <- percent(BRVM_columns$`Market Cap (%)` / 100)

BRVM_columns

In [ ]:
# Sort dataframe in ascending order by a specific column
BRVM_columns_asc <- arrange(BRVM_columns, Date)

# Sort dataframe in descending order by a specific column
BRVM_columns_desc <- arrange(BRVM_columns, desc(Date))

In [ ]:
# Sort dataframe in descending 
BRVM_columns_desc

In [ ]:
# Calculate the length of the tickers list
length(tickers)

In [ ]:
# Select the first 40 rows of BRVM_columns_desc to create Realtime_BRVM
Realtime_BRVM <- head(BRVM_columns_desc, 40)

# Print a message indicating the display of Realtime BRVM Data Columns
cat('Realtime BRVM Data Columns')

# Print the Realtime_BRVM data frame
Realtime_BRVM

In [ ]:
# Retrieve BRVM index data and store it in brvm_sector
brvm_sector <- BRVM_index()

# Rename the column "Indexes" to "Sector" in the brvm_sector dataframe
brvm_sector <- brvm_sector %>%
  rename(Sector = Indexes)

# Print a message indicating that BRVM Indexes are being loaded
print("Loading BRVM Indexes")

# Print the brvm_sector dataframe
brvm_sector

In [ ]:
# Exporting DataFrame into an Excel file for BRVM Sectors

# Write the DataFrame to an Excel file

write.xlsx(brvm_sector, "/Users/Phases/Desktop/BRVM Project/BRVM Excel R files/BRVM_Sectors.xlsx", overwrite = TRUE)

In [ ]:
# Convert the Date column to character type
Realtime_BRVM$Date <- as.character(Realtime_BRVM$Date)

# Export the DataFrame to an Excel file
write.xlsx(Realtime_BRVM, "/Users/Phases/Desktop/BRVM Project/BRVM Excel R files/Realtime_BRVM.xlsx", overwrite = TRUE)

In [ ]:
# Define the file path for the CSV file
file_path <- "/Users/Phases/Desktop/BRVM Project/BRVM Tableau R files/Tableau_BRVM.csv"

# Check if the file already exists
if (file.exists(file_path)) {
  # If the file exists, remove it
  file.remove(file_path)
}

# Write the new CSV file
write.csv(BRVM_columns_desc, file = file_path, row.names = FALSE)

In [ ]:
# Define the file path for the CSV file
file_path <- "/Users/Phases/Desktop/BRVM Project/BRVM Tableau R files/Tableau_BRVM_Sector.csv"

# Check if the file already exists
if (file.exists(file_path)) {
  # If the file exists, remove it
  file.remove(file_path)
}

# Write the new CSV file
write.csv(brvm_sector, file = file_path, row.names = FALSE)